In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt5Agg


In [2]:
import sys
sys.path.append("/home/aliche/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import os
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import colormaps as mycmps
import signale.tools as tools
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import scipy.signal as scsig
from scipy.stats import sem
from mtspec import mtspec
from signale.tools import locate
from os import path

/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/NeuroTools/analysis.py:34: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to u'Qt5Agg' by the following code:
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages

/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


ImportError: No module named mtspec

# Wavelet Analysis Class

In [4]:
"""
A module which implements the continuous wavelet transform

---------------------------------------------------------
Code released under the BSD 3-clause licence.

Copyright (c) 2012, R W Fearick, University of Cape Town
All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the
following conditions are met:

    * Redistributions of source code must retain the above copyright notice, this list of conditions and
        the following disclaimer.
    * Redistributions in binary form must reproduce the above copyright notice, this list of conditions and
        the following disclaimer in the documentation and/or other materials provided with the distribution.
    * Neither the name of the University of Cape Town nor the names of its contributors may be used to endorse
        or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, 
INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR 
SERVICES;LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
WHETHER IN CONTRACT,STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE
USE OF THIS SOFTWARE,EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
---------------------------------------------------------

Wavelet classes:
Morlet
MorletReal
MexicanHat
Paul2      : Paul order 2
Paul4      : Paul order 4
DOG1       : 1st Derivative Of Gaussian
DOG4       : 4th Derivative Of Gaussian
Haar       : Unnormalised version of continuous Haar transform
HaarW      : Normalised Haar

Usage e.g.
wavelet=Morlet(data, largestscale=2, notes=0, order=2, scaling="log")
 data:  Numeric array of data (float), with length ndata.
        Optimum length is a power of 2 (for FFT)
        Worst-case length is a prime
 largestscale:
        largest scale as inverse fraction of length
        scale = len(data)/largestscalax1.set_xticklabels([e
        smallest scale should be >= 2 for meaningful data
 notes: number of scale intervals per octave
        if notes == 0, scales are on a linear increment
 order: order of wavelet for wavelets with variable order
        [Paul, DOG, ..]
 scaling: "linear" or "log" scaling of the wavelet scale.
        Note that feature width in the scale direction
        is constant on a log scale.
        
Attributes of instance:
wavelet.cwt:       2-d array of Wavelet coefficients, (nscales,ndata)
wavelet.nscale:    Number of scale intervals
wavelet.scales:    Array of scale values
                   Note that meaning of the scale will depend on the family
wavelet.fourierwl: Factor to multiply scale by to get scale
                   of equivalent FFT
                   Using this factor, different wavelet families will
                   have comparable scales

References:
A practical guide to wavelet analysis
C Torrance and GP Compo
Bull Amer Meteor Soc Vol 79 No 1 61-78 (1998)
naming below vaguely follows this.

updates:
(24/2/07):  Fix Morlet so can get MorletReal by cutting out H
(10/04/08): Numeric -> numpy
(25/07/08): log and lin scale increment in same direction!
            swap indices in 2-d coeffiecient matrix
            explicit scaling of scale axis
"""

class Cwt:
    """
    Base class for continuous wavelet transforms
    Implements cwt via the Fourier transform
    Used by subclass which provides the method wf(self,s_omega)
    wf is the Fourier transform of the wavelet function.
    Returns an instance.
    """

    fourierwl=1.00

    def _log2(self, x):
        # utility function to return (integer) log2
        return int( np.log(float(x))/ np.log(2.0)+0.0001 )

    def __init__(self, data, largestscale=1, notes=0, order=2, scaling='linear'):
        """
        Continuous wavelet transform of data

        data:    data in array to transform, length must be power of 2
        notes:   number of scale intervals per octave
        largestscale: largest scale as inverse fraction of length
                 of data array
                 scale = len(data)/largestscale
                 smallest scale should be >= 2 for meaningful data
        order:   Order of wavelet basis function for some families
        scaling: Linear or log
        """
        ndata = len(data)
        self.order=order
        self.scale=largestscale
        self._setscales(ndata,largestscale,notes,scaling)
        self.cwt= np.zeros((self.nscale,ndata), np.complex64)
        omega= np.array(range(0,ndata/2)+range(-ndata/2,0))*(2.0*np.pi/ndata)
        datahat=np.fft.fft(data)
        self.fftdata=datahat
        #self.psihat0=self.wf(omega*self.scales[3*self.nscale/4])
        # loop over scales and compute wvelet coeffiecients at each scale
        # using the fft to do the convolution
        for scaleindex in range(self.nscale):
            currentscale=self.scales[scaleindex]
            self.currentscale=currentscale  # for internal use
            s_omega = omega*currentscale
            psihat=self.wf(s_omega)
            psihat = psihat *  np.sqrt(2.0*np.pi*currentscale)
            convhat = psihat * datahat
            W    = np.fft.ifft(convhat)
            self.cwt[scaleindex,0:ndata] = W 
        return
    
    def _setscales(self,ndata,largestscale,notes,scaling):
        """
        if notes non-zero, returns a log scale based on notes per ocave
        else a linear scale
        (25/07/08): fix notes!=0 case so smallest scale at [0]
        """
        if scaling=="log":
            if notes<=0: notes=1 
            # adjust nscale so smallest scale is 2 
            noctave=self._log2( ndata/largestscale/2 )
            self.nscale=notes*noctave
            self.scales=np.zeros(self.nscale,float)
            for j in range(self.nscale):
                self.scales[j] = ndata/(self.scale*(2.0**(float(self.nscale-1-j)/notes)))
        elif scaling=="linear":
            nmax=ndata/largestscale/2
            self.scales=np.arange(float(2),float(nmax))
            self.nscale=len(self.scales)
        else: raise ValueError, "scaling must be linear or log"
        return
    
    def getdata(self):
        """
        returns wavelet coefficient array
        """
        return self.cwt
    def getcoefficients(self):
        return self.cwt
    def getpower(self):
        """
        returns square of wavelet coefficient array
        """
        return (self.cwt* np.conjugate(self.cwt)).real
    def getscales(self):
        """
        returns array containing scales used in transform
        """
        return self.scales
    def getnscale(self):
        """
        return number of scales
        """
        return self.nscale

# wavelet classes    
class Morlet(Cwt):
    """
    Morlet wavelet
    """
    _omega0=6.0
    fourierwl=4* np.pi/(_omega0+ np.sqrt(2.0+_omega0**2))
    def wf(self, s_omega):
        H= np.ones(len(s_omega))
        n=len(s_omega)
        for i in range(len(s_omega)):
            if s_omega[i] < 0.0: H[i]=0.0
        # !!!! note : was s_omega/8 before 17/6/03
        xhat=0.75112554*( np.exp(-(s_omega-self._omega0)**2/2.0))*H
        return xhat

class MorletReal(Cwt):
    """
    Real Morlet wavelet
    """
    _omega0=5.0
    fourierwl=4* np.pi/(_omega0+ np.sqrt(2.0+_omega0**2))
    def wf(self, s_omega):
        H= np.ones(len(s_omega))
        n=len(s_omega)
        for i in range(len(s_omega)):
            if s_omega[i] < 0.0: H[i]=0.0
        # !!!! note : was s_omega/8 before 17/6/03
        xhat=0.75112554*( np.exp(-(s_omega-self._omega0)**2/2.0)
                         + np.exp(-(s_omega+self._omega0)**2/2.0)
                         - np.exp(-(self._omega0)**2/2.0)
                         + np.exp(-(self._omega0)**2/2.0))
        return xhat
    
class Paul4(Cwt):
    """
    Paul m=4 wavelet
    """
    fourierwl=4* np.pi/(2.*4+1.)
    def wf(self, s_omega):
        n=len(s_omega)
        xhat= np.zeros(n)
        xhat[0:n/2]=0.11268723*s_omega[0:n/2]**4* np.exp(-s_omega[0:n/2])
        #return 0.11268723*s_omega**2*exp(-s_omega)*H
        return xhat

class Paul2(Cwt):
    """
    Paul m=2 wavelet
    """
    fourierwl=4* np.pi/(2.*2+1.)
    def wf(self, s_omega):
        n=len(s_omega)
        xhat= np.zeros(n)
        xhat[0:n/2]=1.1547005*s_omega[0:n/2]**2* np.exp(-s_omega[0:n/2])
        #return 0.11268723*s_omega**2*exp(-s_omega)*H
        return xhat

class Paul(Cwt):
    """
    Paul order m wavelet
    """
    def wf(self, s_omega):
        Cwt.fourierwl=4* np.pi/(2.*self.order+1.)
        m=self.order
        n=len(s_omega)
        normfactor=float(m)
        for i in range(1,2*m):
            normfactor=normfactor*i
        normfactor=2.0**m/ np.sqrt(normfactor)
        xhat= np.zeros(n)
        xhat[0:n/2]=normfactor*s_omega[0:n/2]**m* np.exp(-s_omega[0:n/2])
        #return 0.11268723*s_omega**2*exp(-s_omega)*H
        return xhat

class MexicanHat(Cwt):
    """
    2nd Derivative Gaussian (mexican hat) wavelet
    """
    fourierwl=2.0* np.pi/ np.sqrt(2.5)
    def wf(self, s_omega):
        # should this number be 1/sqrt(3/4) (no pi)?
        #s_omega = s_omega/self.fourierwl
        #print max(s_omega)
        a=s_omega**2
        b=s_omega**2/2
        return a* np.exp(-b)/1.1529702
        #return s_omega**2*exp(-s_omega**2/2.0)/1.1529702

class DOG4(Cwt):
    """
    4th Derivative Gaussian wavelet
    see also T&C errata for - sign
    but reconstruction seems to work best with +!
    """
    fourierwl=2.0* np.pi/ np.sqrt(4.5)
    def wf(self, s_omega):
        return s_omega**4* np.exp(-s_omega**2/2.0)/3.4105319

class DOG1(Cwt):
    """
    1st Derivative Gaussian wavelet
    but reconstruction seems to work best with +!
    """
    fourierwl=2.0* np.pi/ np.sqrt(1.5)
    def wf(self, s_omega):
        dog1= np.zeros(len(s_omega),complex64)
        dog1.imag=s_omega* np.exp(-s_omega**2/2.0)/sqrt(pi)
        return dog1

class DOG(Cwt):
    """
    Derivative Gaussian wavelet of order m
    but reconstruction seems to work best with +!
    """
    def wf(self, s_omega):
        try:
            from scipy.special import gamma
        except ImportError:
            print "Requires scipy gamma function"
            raise ImportError
        Cwt.fourierwl=2* np.pi/ np.sqrt(self.order+0.5)
        m=self.order
        dog=1.0J**m*s_omega**m* np.exp(-s_omega**2/2)/ np.sqrt(gamma(self.order+0.5))
        return dog

class Haar(Cwt):
    """
    Continuous version of Haar wavelet
    """
    #    note: not orthogonal!
    #    note: s_omega/4 matches Lecroix scale defn.
    #          s_omega/2 matches orthogonal Haar
    # 2/8/05 constants adjusted to match artem eim

    fourierwl=1.0#1.83129  #2.0
    def wf(self, s_omega):
        haar= np.zeros(len(s_omega),complex64)
        om = s_omega[:]/self.currentscale
        om[0]=1.0  #prevent divide error
        #haar.imag=4.0*sin(s_omega/2)**2/om
        haar.imag=4.0* np.sin(s_omega/4)**2/om
        return haar

class HaarW(Cwt):
    """
    Continuous version of Haar wavelet (norm)
    """
    #    note: not orthogonal!
    #    note: s_omega/4 matches Lecroix scale defn.
    #          s_omega/2 matches orthogonal Haar
    # normalised to unit power

    fourierwl=1.83129*1.2  #2.0
    def wf(self, s_omega):
        haar= np.zeros(len(s_omega),complex64)
        om = s_omega[:]#/self.currentscale
        om[0]=1.0  #prevent divide error
        #haar.imag=4.0*sin(s_omega/2)**2/om
        haar.imag=4.0* np.sin(s_omega/2)**2/om
        return haar




In [5]:
animalPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3903/LinearTrack_05-11-16_Day3/'
lfpPaths = []
for lfp in tools.locate('*-p*.lfp',animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfps = [pkl.load(open(item,'rb'))for item in lfpPaths]
lfpPaths = sorted(lfpPaths)

In [6]:
rippDB = pd.read_pickle('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat3903Lesion/dataFiles/sleepDB/Rat3903-RipplesTimeDB-SKLRN.pd')
rippDB = rippDB[rippDB.date=='day3']
trsh = rippDB.peak_value.mean()+3*rippDB.peak_value.std()
selG = rippDB[(rippDB.peak_value>1*trsh)&(rippDB.wc==100)]
selS = rippDB[(rippDB.peak_value>1*trsh)&(rippDB.wc==300)]

### Colors & Basics

In [7]:
colors = ["windows blue", "amber", "greyish", "faded green", "dusty purple"]
seabC = sns.xkcd_palette(colors)
seabP = sns.color_palette("Paired")
#sns.palplot(seabP)
#sns.palplot(sns.xkcd_palette(colors))
sns.set_style("ticks")
cmapP = sns.light_palette("muted purple", input="xkcd",as_cmap=True)
purples = sns.light_palette("muted purple", input="xkcd")
cmapC = sns.light_palette((210, 90, 60), input="husl",as_cmap=True)
cyans = sns.light_palette((210, 90, 60), input="husl")

## Ripple Examples

In [8]:
fiz,axarr = pl.subplots(1,10,figsize=[20,3])
for bx in axarr:
    bx.set_axis_off()
for iii,kk in enumerate([11078,10764,11668,10740,11181,9924,11468,8752,11342,11749]):#selG.iterrows():
    item = rippDB.loc[kk]
    bx = axarr[iii]# = pl.Axes(fiz, [0., 0., 1., 1.])
    bx.set_axis_off()
    fiz.add_axes(bx)
    cxx = bx.twinx()
    pl.axis('off')
    wavelet=Morlet
    maxscale=4
    notes=64
    scaling="log" #or "linear"
    #scaling="linear"
    plotpower2d=True
    # set up some data
    Ns=2000
    if item.session == 'PRE':
        lfp = lfps[1]
    else:
        lfp = lfps[0]
    tP = item.t_peak
    t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
    t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
    sig = lfp.signal[t1:t2]
    wsig = lfp.signal_white[t1:t2]
    cw=wavelet(wsig,maxscale,notes,scaling=scaling)
    scales=cw.getscales()     
    cwt=cw.getdata()
    # power spectrum
    pwr=cw.getpower()
    scalespec=np.sum(pwr,axis=1)/scales # calculate scale spectrum
    # scales
    y=3.7*cw.fourierwl*scales
    #for jj in range(pwr.shape[1]):
    #    pwr[:,jj] = np.convolve(pwr[:,jj],scsig.gaussian(15,15),'same')
    bx.pcolormesh(np.linspace(-0.512,0.512,1024),y[::-1],(pwr),cmap = 'viridis')
    bx.axhline(150,linewidth='1.5',color = 'c',linestyle='--')
    bx.set_aspect('auto')
    bx.set_xlim(-0.3,0.3)
    bx.set_ylim(4,250)
    cxx.axhline(sig.min(),alpha=0.001)
    #cxx.plot(np.linspace(-0.512,0.512,1024),sig,c='w',lw=0.9,alpha=0.000085)
    span = sig.max()-sig.min()
    smin = -1*sig.max()
    smax = -1*sig.min()
    cxx.plot(np.linspace(-0.512,0.512,1024),-1*sig,c='w',lw=2.0,alpha=0.85)
    if iii<5:
        cxx.set_ylim(smin-span/4,smax+3*span/4)
    else:
        cxx.set_ylim(smin-span/4,smax+3*span/4)
    cxx.set_xlim(-0.25,0.25)
    #cxx.text(0,smin-span/4,kk,color='w')
    fiz.savefig('/home/chenani/ownCloud/Plots/Lesion-Project/wvSample-FGB-'+str(iii)+'.png',transparent=True,dpi=600)

In [8]:
for kk in np.arange(8164,rippDB.index[-1],10):#rippDB.iterrows():
    item = rippDB.loc[kk]
    wavelet=Morlet
    maxscale=4
    notes=64
    scaling="log" #or "linear"
    #scaling="linear"
    plotpower2d=True
    # set up some data
    Ns=2000
    
    #bx.set_xscale('log')
    if item.session == 'PRE':
        lfp = lfps[1]
    else:
        lfp = lfps[0]
    tP = item.t_peak
    t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
    t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
    sig = lfp.signal[t1:t2]
    wsig = lfp.signal_white[t1:t2]
    cw=wavelet(wsig,maxscale,notes,scaling=scaling)
    scales=cw.getscales()     
    cwt=cw.getdata()
    # power spectrum
    pwr=cw.getpower()
    scalespec=np.sum(pwr,axis=1)/scales # calculate scale spectrum
    # scales
    y=3.7*cw.fourierwl*scales
    spec, freq = mtspec(wsig, 1.0, 1, number_of_tapers=5,quadratic=False,nfft=2048)
    freq*=2e3
    #for jj in range(pwr.shape[1]):
    #    pwr[:,jj] = np.convolve(pwr[:,jj],scsig.gaussian(15,3),'same')
    if (freq[spec.argmax()]-150)**2<30:
        fiz = pl.figure(figsize=[4,6])
        gs  = gridspec.GridSpec(1, 4)
        ax = pl.subplot(gs[:,:2])
        bx = pl.subplot(gs[:,3])
        ax.pcolormesh(np.linspace(-0.512,0.512,1024),y[::-1],(pwr),cmap = 'viridis')
        #################
        bx.plot(spec,freq)
        ax.text(0,50,kk,color='w')
        bx.axhline(150,color ='k')
        ax.axhline(150,color ='w')
        bx.set_ylim([0,250])
        ax.set_ylim([0,250])

IndexError: list index out of range

## Basic lyout

In [9]:
sns.set_context("paper",font_scale=1.4)
figure = pl.figure(figsize = [16,9])
gs1 = gridspec.GridSpec(42, 42)
gs2 = gridspec.GridSpec(42, 42)
gs3 = gridspec.GridSpec(42, 42)
gs4 = gridspec.GridSpec(42, 42)
gs1.update(left=0.1, right=0.9,top=1,bottom=0.75, wspace=0.05)
gs2.update(left=0.1, right=0.9,top=0.75,bottom=0.50, wspace=0.05)
gs3.update(left=0.1, right=0.9,top=0.5,bottom=0.25, wspace=0.05)
gs4.update(left=0.1, right=0.9,top=0.25, bottom=0, wspace=0.05)
##########
ax1 = pl.subplot(gs1[6:,:],frameon=False)
###############
ax2 = pl.subplot(gs2[:,1:8])
ax3 = pl.subplot(gs2[:,9:16],sharey=ax2)
ax4 = pl.subplot(gs2[:,18:30])

In [10]:
im = pl.imread('/home/aliche/ownCloud/Plots/Lesion-Project/Sharp-Waves/Spectrograms/timeLapse/wvSample-FGB-9.png')
ax1.imshow(im,aspect='auto')
ax1yticks = np.union1d(np.arange(127,25,-47),np.linspace(127,265,4))
ax1yticks*=6
ax1.set_yticks(ax1yticks)
ax1.set_yticklabels(['250','','150','','50',''])
ax1.set_xlim(250*6,1800*6)
ax1.set_ylim(6*267,6*25)
#ax1.axes.get_xaxis().set_visible(False)
ax1.set_xticks(np.array([770,946,1111])*2)
ax1.set_xticklabels(['-250','0','+250'])
ax1.set_ylabel('Frequency (Hz)')
ax1.set_xlabel('Time (ms)')
ax1.xaxis.set_label_coords(0.04, -0.17)
ax1.xaxis.tick_bottom()
ax1.yaxis.tick_left()

In [ ]:
## gs1.update(left=0.1, right=0.92,top=1,bottom=0.75, wspace=0.05)

In [50]:
ax1.cla()

In [11]:
rippDB = pd.read_pickle('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/sleepDB/Rat3661-RipplesTimeDB-SKLRN.pd')
rippPSD = pkl.load(open('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/sleepDB/Rat3661rippPSD-mtm.npArr','rb'))
wPSD = pkl.load(open('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/sleepDB/Rat3661whiteRippPSD-mtm.npArr','rb'))
freqz = pkl.load(open('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/freqZ.nparr','rb'))
rippDB = rippDB[rippDB.date=='day3']

In [12]:
ax4.set_yscale('log')
sns.kdeplot(rippDB[['wPC1','wPC2']],cmap = 'Greys',shade=True, shade_lowest=False,ax=ax2)
sns.kdeplot(rippDB[rippDB.wc==300][['wPC1','wPC2']],cmap = cmapC,shade=True, shade_lowest=False,ax=ax3)
sns.kdeplot(rippDB[rippDB.wc==100][['wPC1','wPC2']],cmap = cmapP,shade=True, shade_lowest=False,ax=ax3)
ax2.xaxis.tick_bottom()
ax2.yaxis.tick_left()
ax2.set_xticks([])
ax2.set_yticks([])
ax3.xaxis.set_visible(False)
ax3.yaxis.set_visible(False)
ax4.yaxis.tick_left()
ax4.xaxis.tick_bottom()
ax4.set_yticklabels([1e5,1e6,1e7])
ax4.set_yticklabels(['','',''])
ax4.set_ylim(2e5,2e6)
ax2.set_ylabel('PC2')
ax2.set_xlabel('PC1')
ax4.set_ylabel('PSD (au)')
ax4.set_xlabel('Frequency (Hz)')
sns.tsplot(wPSD[rippDB.index[(rippDB.wc==300).as_matrix()]],time = freqz[:35],color=cyans[-1],ax=ax4)
sns.tsplot(wPSD[rippDB.index[(rippDB.wc==300).as_matrix()]],time = freqz[:35],color=cyans[-1],ci=99.9,err_style="ci_bars",marker=',', interpolate=False,ax=ax4)
sns.tsplot(wPSD[rippDB.index[(rippDB.wc==100).as_matrix()]],time = freqz[:35],ax=ax4,color=purples[-1])
sns.tsplot(wPSD[rippDB.index[(rippDB.wc==100).as_matrix()]],time = freqz[:35],color=purples[-1],ci=99.9,err_style="ci_bars",marker=',', interpolate=False,ax=ax4)
#pl.scatter(rippDB[rippDB.wc==300][['wPC1']],rippDB[rippDB.wc==300][['wPC2']],c='#00468b',marker='x',alpha=0.25)
#pl.scatter(rippDB[rippDB.wc==100][['wPC1']],rippDB[rippDB.wc==100][['wPC2']],c='#8b0000',marker='x',alpha=0.25)

/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/seaborn/distributions.py:645: UserWarning: Passing a 2D dataset for a bivariate plot is deprecated in favor of kdeplot(x, y), and it will cause an error in future versions. Please update your code.
  warnings.warn(warn_msg, UserWarning)
/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/seaborn/timeseries.py:183: UserWarning: The tsplot function is deprecated and will be removed or replaced (in a substantially altered version) in a future release.
  warnings.warn(msg, UserWarning)


In [13]:
gs2.update(left=0.08, right=0.92,top=0.68,bottom=0.46, wspace=0.05)
gs3.update(left=0.1, right=0.9,top=0.45,bottom=0.25, wspace=0.05)

In [296]:
ax2.cla()
ax3.cla()
ax4.cla()

In [27]:
ax5 = pl.subplot(gs2[:,33:37])
ax6 = pl.subplot(gs2[:,38:42],sharey=ax5)
taxis = np.linspace(-100,100,20)
controlRates = []
lesionRates = []
shuffledL = []
shuffledC = []
for pth in sorted([os.path.join(item[0],item[1]) for item in tools.locate('*SWR_mean.npArr','/home/aliche/ownCloud/IpythonNotebooks/')]):
    mtrx = pkl.load(open(pth,'rb'))
    if pth.find('ct/cont')>0:
        for ii in range(len(mtrx)):
            controlRates.append(mtrx[ii])
    else:
        for ii in range(len(mtrx)):
            lesionRates.append(mtrx[ii])
for pth in sorted([os.path.join(item[0],item[1]) for item in tools.locate('*RND_mean.npArr','/home/aliche/ownCloud/IpythonNotebooks/')]):
    mtrx = pkl.load(open(pth,'rb'))
    shuffledL.append(mtrx[ii])
sns.tsplot(controlRates,time=taxis,interpolate=False,marker='.',color=cyans[-1],err_style="ci_bars",ax=ax5)
sns.tsplot(lesionRates,time=taxis,interpolate=False,marker='.',color=cyans[-1],err_style="ci_bars",ax=ax6)
#sns.tsplot(shuffledL,time=taxis,interpolate=False,color='0.7',err_style="ci_bars",ax=ax5,zorder=-1)
#handles, labels = ax.get_legend_handles_labels()
#ax.legend()
ax5.set_ylabel('pop. rate (z)')
ax5.set_xlabel('Time (ms)')
ax5.set_title('Control')
ax5.set_ylim([-0.03,1.7])
ax5.xaxis.tick_bottom()
ax5.yaxis.tick_left()
###############################################################################################
controlRates = []
lesionRates = []
for pth in sorted([os.path.join(item[0],item[1]) for item in tools.locate('*FGB_mean.npArr','/home/aliche/ownCloud/IpythonNotebooks/')]):
    mtrx = pkl.load(open(pth,'rb'))
    if pth.find('ct/cont')>0:
        for ii in range(len(mtrx)):
            controlRates.append(mtrx[ii])
    else:
        for ii in range(len(mtrx)):
            lesionRates.append(mtrx[ii])
for pth in sorted([os.path.join(item[0],item[1]) for item in tools.locate('*RND_mean.npArr','/home/aliche/ownCloud/IpythonNotebooks/')]):
    mtrx = pkl.load(open(pth,'rb'))
    shuffledL.append(mtrx[ii])
sns.tsplot(controlRates,time=taxis,interpolate=False,marker='.',color=purples[-1],err_style="ci_bars",ax=ax5)
sns.tsplot(lesionRates,time=taxis,interpolate=False,marker='.',color=purples[-1],err_style="ci_bars",ax=ax6)
#sns.tsplot(shuffledL,time=taxis,interpolate=False,err_style="ci_bars",ax=ax6)
#ax6.set_xlabel('Time (ms)')
ax6.set_title('Lesioned')
ax6.yaxis.tick_right()
ax6.xaxis.tick_bottom()
ax6.set_xticklabels(['','','','',''])
#pl.tight_layout()


[Text(0,0,u''), Text(0.5,0,u''), Text(1,0,u''), Text(0,0,u''), Text(0,0,u'')]

In [26]:
ax5.cla()
ax6.cla()

In [25]:
ax55 = pl.subplot(gs3[:20,:7])
ax66 = pl.subplot(gs3[:20,8:15],sharey=ax55,sharex=ax55)
ax77 = pl.subplot(gs3[22:,:7],sharey=ax55,sharex=ax55)
ax88 = pl.subplot(gs3[22:,8:15],sharey=ax55,sharex=ax55)

In [5]:
patCCDF = pd.read_pickle('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/patCCDF.db')
syncPCdb = pd.DataFrame([],columns=['anID','group','patID','session','hfe_type','prcnt'])
for anID in np.unique(patCCDF.anDate):
    df = patCCDF[patCCDF.anDate==anID]
    for ptID in range(1,10):
        for sess in ['PRE','POST']:
            dfSessID = df[(df.session==sess)&(df.ptIdx==ptID)]
            if dfSessID.shape[0]:
                for typ in ['swr','fgb']:
                    dfTyp = dfSessID[dfSessID['type']==typ]
                    if dfTyp.shape[0]:
                        syncPCdb = syncPCdb.append(pd.DataFrame([[anID,np.unique(dfTyp.group)[0],ptID,sess,typ,
                                                                  1e2*dfTyp[dfTyp.ccdf<50].shape[0]/dfSessID.shape[0]]]
                                                  ,columns=['anID','group','patID','session','hfe_type','prcnt']),ignore_index=True)


In [19]:
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='PRE')&(syncPCdb.hfe_type=='swr')],
           interpolate=True,err_style="boot_traces",n_boot=50,estimator=np.nanmean,color=['r','k'],ax=ax55)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='PRE')&(syncPCdb.hfe_type=='swr')],
           interpolate=False,err_style="ci_bars",estimator=np.nanmean,color=['r','k'],ax=ax55)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='POST')&(syncPCdb.hfe_type=='swr')],
           interpolate=True,err_style="boot_traces",n_boot=50,estimator=np.nanmean,color=['r','k'],ax=ax66)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='POST')&(syncPCdb.hfe_type=='swr')],
           interpolate=False,err_style="ci_bars",estimator=np.nanmean,color=['r','k'],ax=ax66)
ax55.legend([])
ax66.legend([])
ax55.xaxis.tick_top()
ax66.xaxis.tick_top()
ax55.yaxis.tick_left()
ax66.yaxis.tick_right()
ax55.set_yticks([10,30,50])
ax55.set_xticks(range(1,10))
#ax55.set_yticklabels(['','0',''])
ax55.set_xlabel('')
ax66.set_xlabel('')
ax55.set_title('PRE')
ax66.set_title('POST')
ax55.set_ylabel('%$A_{peak}$ sync. SWR')
ax66.set_ylabel('')
pl.setp(ax55.get_xticklabels(), visible=False);
pl.setp(ax55.get_xticklines(), visible=False);
pl.setp(ax66.get_xticklabels(), visible=False);
pl.setp(ax66.get_xticklines(), visible=False);
pl.setp(ax66.get_yticklabels(), visible=False);
pl.setp(ax66.get_yticklines(), visible=False);

In [20]:
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='PRE')&(syncPCdb.hfe_type=='fgb')],
           interpolate=True,err_style="boot_traces",n_boot=50,estimator=np.nanmean,color=['r','k'],ax=ax77)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='PRE')&(syncPCdb.hfe_type=='fgb')],
           interpolate=False,err_style="ci_bars",estimator=np.nanmean,color=['r','k'],ax=ax77)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='POST')&(syncPCdb.hfe_type=='fgb')],
           interpolate=True,err_style="boot_traces",n_boot=50,estimator=np.nanmean,color=['r','k'],ax=ax88)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='POST')&(syncPCdb.hfe_type=='fgb')],
           interpolate=False,err_style="ci_bars",estimator=np.nanmean,color=['r','k'],ax=ax88)
ax77.legend([])
ax88.legend([])
ax77.xaxis.tick_bottom()
ax88.xaxis.tick_bottom()
ax77.yaxis.tick_left()
ax88.yaxis.tick_right()
ax77.set_yticks([10,30,50])
ax77.set_xlabel('Pattern index.')
ax88.set_xlabel('')
ax77.set_ylabel('%$A_{peak}$ sync. FGB')
ax88.set_ylabel('')
pl.setp(ax88.get_yticklabels(), visible=False);
pl.setp(ax88.get_yticklines(), visible=False);

In [99]:
ax55.cla()
ax66.cla()
ax77.cla()
ax88.cla()

In [21]:
gs3.update(left=0.1, right=0.94,top=0.4,bottom=0.07, wspace=0.05)

In [22]:
ax7 = pl.subplot(gs3[:,13:17])
#ax8 = pl.subplot(gs3[:,4:8])
#ax9 = pl.subplot(gs3[:,8:12])
fgRipp_percentiles = pd.read_pickle('/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/RippPercentileDF.db')
#sns.violinplot(x='group',y='SWR_prcnt',data=fgRipp_percentiles,inner='stick',split=True,hue='session',order=['control','lesion'],palette=np.array(cyans)[[0,-1]],linewidth=2,ax=ax7)
#sns.violinplot(x='group',y='SWR_sws',data=fgRipp_percentiles,inner='stick',split=True,hue='session',order=['control','lesion'],palette=np.array(cyans)[[0,-1]],linewidth=2,ax=ax8)
#sns.violinplot(x='group',y='SWR_rem',data=fgRipp_percentiles,inner='stick',split=True,hue='session',order=['control','lesion'],palette=np.array(cyans)[[0,-1]],linewidth=2,ax=ax9)
sns.boxplot(x='group',y='SWR_prcnt',hue='session',data=fgRipp_percentiles,linewidth=1,order=['control','lesion'],hue_order=['PRE','POST'],fliersize=0.1,palette=np.array(cyans)[[0,-1]],ax=ax7)
#sns.swarmplot(x='group',y='SWR_prcnt',hue='session',size = 4,split=True,order=['control','lesion'],hue_order=['PRE','POST'],data=fgRipp_percentiles,palette=np.array(cyans)[[2,-3]],ax=ax7)
#handles, labels = ax.get_legend_handles_labels()
ax7.text(-0.15, 80, r'**', fontsize=20)
ax7.text(0.85, 84, r'n.s.', fontsize=15)
ax7.text(0.3, 101, r'***', fontsize=20)
#ax8.text(0.4, 97, r'***', fontsize=20)
#ax9.text(0.4, 97, r'***', fontsize=20)
ax7.plot([0,1],[100,100],'k',lw=3)
ax7.plot([0.8,1.2],[82,82],'k',lw=3)
ax7.plot([-0.2,0.2],[82,82],'k',lw=3)
#ax7.annotate('', xy=(0, 91.), xycoords='data',xytext=(1., 91),arrowprops=dict(arrowstyle="-",connectionstyle="bar",ec="k",shrinkA=5, shrinkB=5))
#ax7.annotate('', xy=(0, 33.), xycoords='data',xytext=(1., 91),arrowprops=dict(arrowstyle="-",connectionstyle="bar",ec="k",shrinkA=5, shrinkB=5))
#ax8.annotate('', xy=(0, 91.), xycoords='data',xytext=(1., 91),arrowprops=dict(arrowstyle="-",connectionstyle="bar",ec="k",shrinkA=5, shrinkB=5))
#ax9.annotate('', xy=(0, 91.), xycoords='data',xytext=(1., 91),arrowprops=dict(arrowstyle="-",connectionstyle="bar",ec="k",shrinkA=5, shrinkB=5))
#ax8.text(0.04, 90, r'**', fontsize=20)
#ax8.text(1.04, 90, r'n.s.', fontsize=15)
#ax9.text(0.04, 90, r'n.s.', fontsize=15)
#ax9.text(1.04, 90, r'n.s.', fontsize=15)
#ax8.legend('')
#ax9.legend('')
ax7.legend(bbox_to_anchor=(4.9,1.04),prop={'size':9})
ax7.set_title('SWR ratio')
ax7.set_xlabel('')
ax7.set_xticklabels(['Cnt','Les'],fontsize=14,rotation=0, rotation_mode="anchor",ha='center')
#ax8.set_xticklabels(['',''],fontsize=14,rotation=-25, rotation_mode="anchor",ha='center')
#ax9.set_xticklabels(['',''],fontsize=14,rotation=-25, rotation_mode="anchor",ha='center')
#ax8.set_title('DELTA')
#ax8.set_xlabel('')
#ax9.set_title('THETA')
#ax9.set_xlabel('')
ax7.set_ylabel('%')
#ax8.set_ylabel('')
#ax9.set_ylabel('')
#ax8.set_yticks([])
#ax9.set_yticks([])
ax7.yaxis.tick_left()
ax7.xaxis.tick_bottom()
#ax8.xaxis.tick_bottom()
#ax9.xaxis.tick_bottom()
ax7.set_ylim(-1,120)
#ax8.set_ylim(-1,120)
#ax9.set_ylim(-1,120)
#ax8.yaxis.set_clip_on(1)
ax7.set_yticks(np.arange(0,120,25))
#ax7.set_yticklabels(['0','','40','','80',''])
######
for i,artist in enumerate(ax7.artists):
    # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    col = artist.get_facecolor()
    artist.set_edgecolor(kr[i])

    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    for j in range(i*6,i*6+6):
        line = ax7.lines[j]
        line.set_color(kr[i])
        line.set_mfc(kr[i])
        line.set_mec(kr[i])

NameError: name 'kr' is not defined

In [17]:
ax7.cla()
#ax8.cla()
#ax9.cla()

In [17]:
fg10th = fgRipp_percentiles.copy()
fg10th.SWR_prcnt /=20.0
lesFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/'
cntFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/'
lesListPlus_pre = sorted([path.join(item[0],item[1]) for item in locate('*4plus-PRE.npArr',lesFolder)])
lesListPlus_post = sorted([path.join(item[0],item[1]) for item in locate('*4plus-POST.npArr',lesFolder)])
cntListPlus_pre = sorted([path.join(item[0],item[1]) for item in locate('*4plus-PRE.npArr',cntFolder)])
cntListPlus_post = sorted([path.join(item[0],item[1]) for item in locate('*4plus-POST.npArr',cntFolder)])
#########################
lesListMinus = sorted([path.join(item[0],item[1]) for item in locate('*4minus-PRE.npArr',lesFolder)])
cntListMinus = sorted([path.join(item[0],item[1]) for item in locate('*4minus-PRE.npArr',cntFolder)])
#########################
lesListSig_pre = sorted([path.join(item[0],item[1]) for item in locate('*PRE-Csig.npArr',lesFolder)])
lesListSig_post = sorted([path.join(item[0],item[1]) for item in locate('*POST-Csig.npArr',lesFolder)])
cntListSig_pre = sorted([path.join(item[0],item[1]) for item in locate('*PRE-Csig.npArr',cntFolder)])
cntListSig_post= sorted([path.join(item[0],item[1]) for item in locate('*POST-Csig.npArr',cntFolder)])
#lesListSig_post
popPrcntDB = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/files/pop-SWR-prcnt-NoSig.db')

In [18]:
ax10a = pl.subplot(gs3[:,19:23],sharey=ax7)
sns.boxplot(x='group',y='prcnt',data=popPrcntDB,hue='session',linewidth=1,order=['control','lesion'],hue_order=['PRE','POST'],fliersize=0.1,palette=np.array(cyans)[[0,-1]],ax=ax10a)
#ax10a.set_ylim(30,105)
ax10a.legend([])
ax10a.set_yticks([0,50,100])
ax10a.xaxis.tick_bottom()
ax10a.yaxis.tick_left()
ax10a.set_ylabel('%')
ax10a.set_xlabel('')
ax10a.set_ylim(-1,120)
#####
kr = ['k','k','r','r']
for i,artist in enumerate(ax10a.artists):
    # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    col = artist.get_facecolor()
    artist.set_edgecolor(kr[i])

    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    for j in range(i*6,i*6+6):
        line = ax10a.lines[j]
        line.set_color(kr[i])
        line.set_mfc(kr[i])
        line.set_mec(kr[i])
############################
ax10b = pl.subplot(gs3[:,25:29])
sns.boxplot(x='group',y='Rpop',data=popPrcntDB,hue='session',linewidth=1,order=['control','lesion'],hue_order=['PRE','POST'],fliersize=0.1,palette=np.array(cyans)[[0,-1]],ax=ax10b)
#ax10a.set_ylim(30,105)
ax10b.legend([])
ax10b.set_yticks([0.1,0.3,0.5])
ax10b.xaxis.tick_bottom()
ax10b.yaxis.tick_left()
ax10b.set_ylabel('Rate (Hz)')
ax10b.set_xlabel('')
ax10b.yaxis.set_label_position('left')
#ax10b.yaxis.set_label_coords(1.4,0.5)
ax10b.set_ylim(-0.01,0.6)
######
for i,artist in enumerate(ax10b.artists):
    # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    col = artist.get_facecolor()
    artist.set_edgecolor(kr[i])

    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    for j in range(i*6,i*6+6):
        line = ax10b.lines[j]
        line.set_color(kr[i])
        line.set_mfc(kr[i])
        line.set_mec(kr[i])

In [442]:
ax10a.cla()
ax10b.cla()

In [19]:
gs4.update(left=0.1, right=0.9,top=0.19,bottom=0.04, wspace=0.05)

In [20]:
ax10 = pl.subplot(gs4[:,:4])
swrPrcnt = pd.DataFrame([],columns=['prcnt','group','session'])
lst = [cntListPlus_pre,cntListPlus_post,lesListPlus_pre,lesListPlus_post]
sess = ['PRE','POST','PRE','POST']
grp  = ['control','control','lesion','lesion'] 
for idx,pthLst in enumerate(lst):
    for ccFile in pthLst:
        dayList = pkl.load(open(ccFile,'rb'))
        for dic in dayList:
            swrs = dic['swr']
            fgbs = dic['fgb']
            if swrs.size+fgbs.size:
                swrPrcnt = swrPrcnt.append(pd.DataFrame([[1e2*swrs.size/(fgbs.size+swrs.size),grp[idx],sess[idx]]],columns=['prcnt','group','session']),ignore_index=True)

####################
#sns.violinplot(x='SWR_prcnt',y='group',data=fgRipp_percentiles,inner=None,split=True,hue='session',orient='h',order=['control','lesion'],hue_order=['POST','PRE'],alpha=0.5,ax=ax)
sns.boxplot(x='group',y='SWR_prcnt',data=fgRipp_percentiles,hue='session',orient='v',linewidth=1,order=['control','lesion'],hue_order=['PRE','POST'],fliersize=0.1,palette=np.array(cyans)[[0,-1]],ax=ax10)
sns.swarmplot(x='group',y='prcnt',data=swrPrcnt,split=True,hue='session',orient='v',size=4,order=['control','lesion'],hue_order=['PRE','POST'],palette=np.array(cyans)[[2,-3]],ax=ax10);
ax10.legend([])#bbox_to_anchor=(1.4, 0.5))
#pl.yticks([0,50,100])
ax10.set_title('$\pm 150$ ms')
ax10.set_xlabel('')
ax10.set_ylabel('')
ax10.set_yticks([0,50,100])
ax10.xaxis.tick_bottom()
ax10.yaxis.tick_left()
#pl.tight_layout()
#f.savefig('/home/chenani/ownCloud/Plots/Lesion-Project/SWR-Coincidence-percentile.pdf')
#####
kr = ['k','k','r','r']
for i,artist in enumerate(ax10.artists):
    # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    col = artist.get_facecolor()
    artist.set_edgecolor(kr[i])

    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    for j in range(i*6,i*6+6):
        line = ax10.lines[j]
        line.set_color(kr[i])
        line.set_mfc(kr[i])
        line.set_mec(kr[i])

In [254]:
ax10.cla()

In [21]:
ax11 = pl.subplot(gs4[:,4:8],sharey=ax10a)
syncPrcnt = pd.DataFrame([],columns=['prcnt','session','group','event'])
syncLag = 0.0075
lst = [cntListPlus_pre,cntListPlus_post,lesListPlus_pre,lesListPlus_post]
sess = ['PRE','POST','PRE','POST']
grp  = ['control','control','lesion','lesion'] 
for idx,pthLst in enumerate(lst):
    for ccFile in pthLst:
        dayList = pkl.load(open(ccFile,'rb'))
        for dic in dayList:
            swrs = dic['swr']
            fgbs = dic['fgb']
            syncSWR = swrs[np.abs(swrs)<syncLag]
            syncFGB = fgbs[np.abs(fgbs)<syncLag]
            if syncFGB.size+syncSWR.size:
                syncPrcnt = syncPrcnt.append(pd.DataFrame([[1e2*syncSWR.size/(syncFGB.size+syncSWR.size),sess[idx],grp[idx],'SWR']],columns=['prcnt','session','group','event']),ignore_index=True)
                syncPrcnt = syncPrcnt.append(pd.DataFrame([[1e2*syncFGB.size/(syncFGB.size+syncSWR.size),sess[idx],grp[idx],'FGB']],columns=['prcnt','session','group','event']),ignore_index=True)
####################
#sns.violinplot(x='SWR_prcnt',y='group',data=fgRipp_percentiles,inner=None,split=True,hue='session',orient='h',order=['control','lesion'],hue_order=['POST','PRE'],alpha=0.5,ax=ax)
sns.boxplot(x='group',y='SWR_prcnt',data=fgRipp_percentiles,hue='session',orient='v',linewidth=1,order=['control','lesion'],hue_order=['PRE','POST'],fliersize=0.1,palette=np.array(cyans)[[0,-1]],ax=ax11)
sns.swarmplot(x='group',y='prcnt',data=syncPrcnt[syncPrcnt.event=='SWR'],split=True,hue='session',orient='v',size=4,order=['control','lesion'],hue_order=['PRE','POST'],palette=np.array(cyans)[[2,-3]],ax=ax11);
ax11.legend([])#bbox_to_anchor=(1.4, 0.5))
#pl.yticks([0,50,100])
ax11.set_title('$\pm 7$ ms')
ax11.set_xlabel('')
ax11.set_ylabel('')
ax11.set_yticks([0,50,100])
ax11.xaxis.tick_bottom()
ax11.yaxis.tick_right()
#####
kr = ['k','k','r','r']
for i,artist in enumerate(ax11.artists):
    # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    col = artist.get_facecolor()
    artist.set_edgecolor(kr[i])

    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    for j in range(i*6,i*6+6):
        line = ax11.lines[j]
        line.set_color(kr[i])
        line.set_mfc(kr[i])
        line.set_mec(kr[i])

In [252]:
ax11.cla()

In [22]:
ax12 = pl.subplot(gs4[:,12:16],sharey=ax10a)
swrPrcntSig = pd.DataFrame([],columns=['prcnt','group','session'])
lst = [cntListSig_pre,cntListSig_post,lesListSig_pre,lesListSig_post]
sess = ['PRE','POST','PRE','POST']
grp  = ['control','control','lesion','lesion'] 
for idx,pthLst in enumerate(lst):
    for ccFile in pthLst:
        dayList = pkl.load(open(ccFile,'rb'))
        for dic in dayList:
            swrs = dic['swr']
            fgbs = dic['fgb']
            if swrs.size+fgbs.size:
                #print swrs.size
                swrPrcntSig = swrPrcntSig.append(pd.DataFrame([[1e2*swrs.size/(fgbs.size+swrs.size),grp[idx],sess[idx]]],columns=['prcnt','group','session']),ignore_index=True)

####################
#sns.violinplot(x='SWR_prcnt',y='group',data=fgRipp_percentiles,inner=None,split=True,hue='session',orient='h',order=['control','lesion'],hue_order=['POST','PRE'],alpha=0.5,ax=ax)
sns.boxplot(x='group',y='SWR_prcnt',data=fgRipp_percentiles,hue='session',orient='v',linewidth=1,order=['control','lesion'],hue_order=['PRE','POST'],fliersize=0.1,palette=np.array(cyans)[[0,-1]],ax=ax12)
sns.swarmplot(x='group',y='prcnt',data=swrPrcntSig,split=True,hue='session',orient='v',size=4,order=['control','lesion'],hue_order=['PRE','POST'],palette=np.array(cyans)[[2,-3]],ax=ax12);
ax12.legend([])#bbox_to_anchor=(1.4, 0.5))
#pl.yticks([0,50,100])
ax12.set_title('$\pm 150$ ms')
ax12.set_xlabel('')
ax12.set_ylabel('')
ax12.set_yticks([0,50,100])
ax12.xaxis.tick_bottom()
ax12.yaxis.tick_left()
#####
for i,artist in enumerate(ax12.artists):
    # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    col = artist.get_facecolor()
    artist.set_edgecolor(kr[i])

    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    for j in range(i*6,i*6+6):
        line = ax12.lines[j]
        line.set_color(kr[i])
        line.set_mfc(kr[i])
        line.set_mec(kr[i])

In [258]:
ax12.cla()

In [23]:
ax13 = pl.subplot(gs4[:,16:20])
syncPrcntSig = pd.DataFrame([],columns=['prcnt','session','group','event'])
syncLag = 0.0075
lst = [cntListSig_pre,cntListSig_post,lesListSig_pre,lesListSig_post]
sess = ['PRE','POST','PRE','POST']
grp  = ['control','control','lesion','lesion'] 
for idx,pthLst in enumerate(lst):
    for ccFile in pthLst:
        dayList = pkl.load(open(ccFile,'rb'))
        for dic in dayList:
            swrs = dic['swr']
            fgbs = dic['fgb']
            syncSWR = swrs[np.abs(swrs)<syncLag]
            syncFGB = fgbs[np.abs(fgbs)<syncLag]
            if syncFGB.size+syncSWR.size:
                syncPrcntSig = syncPrcntSig.append(pd.DataFrame([[1e2*syncSWR.size/(syncFGB.size+syncSWR.size),sess[idx],grp[idx],'SWR']],columns=['prcnt','session','group','event']),ignore_index=True)
                syncPrcntSig = syncPrcntSig.append(pd.DataFrame([[1e2*syncFGB.size/(syncFGB.size+syncSWR.size),sess[idx],grp[idx],'FGB']],columns=['prcnt','session','group','event']),ignore_index=True)
sns.boxplot(x='group',y='SWR_prcnt',data=fgRipp_percentiles,hue='session',orient='v',linewidth=1,order=['control','lesion'],hue_order=['PRE','POST'],palette=np.array(cyans)[[0,-1]],fliersize=0.1,ax=ax13)
sns.swarmplot(x='group',y='prcnt',data=syncPrcntSig[syncPrcntSig.event=='SWR'],split=True,hue='session',orient='v',size=4,order=['control','lesion'],hue_order=['PRE','POST'],palette=np.array(cyans)[[2,-3]],ax=ax13);
ax13.legend([])#bbox_to_anchor=(1.4, 0.5))
#pl.yticks([0,50,100])
ax13.set_title('$\pm 7$ ms')
ax13.set_xlabel('')
ax13.set_ylabel('')
ax13.set_yticks([0,50,100])
ax13.xaxis.tick_bottom()
ax13.yaxis.tick_right()
#####
for i,artist in enumerate(ax13.artists):
    # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    col = artist.get_facecolor()
    artist.set_edgecolor(kr[i])

    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    for j in range(i*6,i*6+6):
        line = ax13.lines[j]
        line.set_color(kr[i])
        line.set_mfc(kr[i])
        line.set_mec(kr[i])

In [261]:
ax13.cla()

In [26]:
from scipy.io import savemat
savemat('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/SWR_prcnt.mat',fgRipp_percentiles[['group','session','FG_prcnt','SWR_prcnt']].to_dict(orient='list'))

In [25]:
fgRipp_percentiles[['group','session','FG_prcnt','SWR_prcnt']]

,group,session,FG_prcnt,SWR_prcnt
0,lesion,PRE,52.943598,47.056402
1,lesion,POST,52.932442,47.067558
2,lesion,POST,76.463878,23.536122
3,lesion,PRE,46.083597,53.916403
4,lesion,PRE,75.686431,24.313569
5,lesion,POST,72.560733,27.439267
6,lesion,PRE,99.600436,0.399564
7,lesion,POST,98.517110,1.482890
8,lesion,PRE,70.535714,11.734694
9,lesion,POST,76.555024,14.641148


In [6]:
lesPat = syncPCdb[syncPCdb.group=='lesion']
cntPat = syncPCdb[syncPCdb.group=='control']
cntPat.patID = np.int8(cntPat.patID)
lesPat.patID = np.int8(lesPat.patID)
import statsmodels.api as sm
from statsmodels.formula.api import ols
model = ols('prcnt~C(hfe_type)*C(session)+patID',data=cntPat).fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                  prcnt   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     60.10
Date:                 Mi, 09 Mai 2018   Prob (F-statistic):           6.86e-42
Time:                        13:25:19   Log-Likelihood:                -2086.8
No. Observations:                 538   AIC:                             4184.
Df Residuals:                     533   BIC:                             4205.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [97]:
model.model.data.orig_exog[:5]

,Intercept,C(hfe_type)[T.swr],C(session)[T.PRE],C(hfe_type)[T.swr]:C(session)[T.PRE],patID
108,1.0,1.0,1.0,1.0,1.0
109,1.0,0.0,1.0,0.0,1.0
110,1.0,1.0,0.0,0.0,1.0
111,1.0,0.0,0.0,0.0,1.0
112,1.0,1.0,1.0,1.0,2.0


In [98]:
model.model.data.frame[:5]

,anID,group,patID,session,hfe_type,prcnt
108,3.0,control,1,PRE,swr,46.250000
109,3.0,control,1,PRE,fgb,20.625000
110,3.0,control,1,POST,swr,75.280899
111,3.0,control,1,POST,fgb,7.303371
112,3.0,control,2,PRE,swr,10.460251


In [7]:
symbols = ['D', '^']
colors = ['r', 'g', 'b']
syncPCdb.patID = np.int8(syncPCdb.patID)
factor_groups = syncPCdb.groupby(['session','hfe_type','group'])
infl = model.get_influence()
df_infl = infl.summary_frame()
resid = model.resid
pl.figure(figsize=(6,6));
for _ii_,_item_ in enumerate(factor_groups):
    print _item_[0]
    group = _item_[1]
    lm = ols('prcnt~patID',data=group).fit()
    print anova_lm(lm)
    #print _ii_,values
    print lm.pvalues
    print '****************************************************************'

('POST', 'fgb', 'control')


NameError: name 'anova_lm' is not defined

In [9]:
factor_groups = syncPCdb.groupby(['hfe_type','group'])
infl = model.get_influence()
df_infl = infl.summary_frame()
resid = model.resid
for _ii_,_item_ in enumerate(factor_groups):
    print _item_[0]
    group = _item_[1]
    print group.shape
    lm = ols('prcnt~C(session)',data=group).fit()
    #print _ii_,values
    print lm.pvalues
    print '****************************************************************'

('fgb', 'control')
(268, 6)
Intercept            6.141247e-45
C(session)[T.PRE]    4.539556e-06
dtype: float64
****************************************************************
('fgb', 'lesion')
(333, 6)
Intercept            3.057125e-47
C(session)[T.PRE]    5.309315e-01
dtype: float64
****************************************************************
('swr', 'control')
(270, 6)
Intercept            1.854526e-55
C(session)[T.PRE]    3.052546e-09
dtype: float64
****************************************************************
('swr', 'lesion')
(327, 6)
Intercept            5.024704e-29
C(session)[T.PRE]    1.008921e-01
dtype: float64
****************************************************************


In [10]:
factor_groups = syncPCdb.groupby(['session','hfe_type'])
infl = model.get_influence()
df_infl = infl.summary_frame()
resid = model.resid
for _ii_,_item_ in enumerate(factor_groups):
    print _item_[0]
    group = _item_[1]
    print group.shape
    lm = ols('prcnt~C(group)',data=group).fit()
    #print _ii_,values
    print lm.pvalues
    print '****************************************************************'

('POST', 'fgb')
(296, 6)
Intercept             3.701462e-36
C(group)[T.lesion]    3.856112e-04
dtype: float64
****************************************************************
('POST', 'swr')
(295, 6)
Intercept             1.909256e-62
C(group)[T.lesion]    1.132073e-21
dtype: float64
****************************************************************
('PRE', 'fgb')
(305, 6)
Intercept             5.484532e-47
C(group)[T.lesion]    8.636103e-01
dtype: float64
****************************************************************
('PRE', 'swr')
(302, 6)
Intercept             9.405447e-46
C(group)[T.lesion]    7.876668e-10
dtype: float64
****************************************************************


In [123]:
from statsmodels.stats.api import anova_lm

In [121]:
anova_lm(lm).keys()

Index([u'df', u'sum_sq', u'mean_sq', u'F', u'PR(>F)'], dtype='object')